## **Thiết lập môi trường**

In [ ]:
# Kết nối Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# ! wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
! tar xf spark-3.5.4-bin-hadoop3.tgz
! pip install -q findspark

In [ ]:
!mv spark-3.5.4-bin-hadoop3.tgz /content/drive/MyDrive/MMDS/spark/

In [ ]:
! du -sh spark-3.5.4-bin-hadoop3.tgz

383M	spark-3.5.4-bin-hadoop3.tgz


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

In [ ]:
import findspark
import pyspark as spark

findspark.init()
print(spark.__version__)

3.5.5


# **Working Path**

In [ ]:
working_path = "/content/drive/MyDrive/MMDS"

**Các thư viện cần thiết**

In [ ]:
from pyspark.sql.functions import col, expr, collect_set, udf, to_date
from pyspark.ml.feature import CountVectorizer, MinHashLSH
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession
from tabulate import tabulate
import matplotlib.pyplot as plt
import time


# **Brute-force Algorithm**





In [ ]:
class BruteForceApproach:
    def __init__(self, file_path):
        """Khởi tạo SparkSession, đọc dữ liệu và tiền xử lý."""
        self.spark = SparkSession.builder.appName("BruteForceApproach").getOrCreate()
        self.file_path = file_path
        self.df = self.spark.read.csv(self.file_path, header=True, inferSchema=True)
        self.df = self.df.withColumn("Date", to_date(col("Date"), "dd/MM/yyyy"))

    def pre_process(self):
        # Tiền xử lý: Nhóm sản phẩm theo ngày
        self.transactions = self.df.groupBy("Date").agg(
            collect_set("itemDescription").alias("items")
        )

    @staticmethod
    def compute_jaccard(set1, set2):
        """Tính Jaccard Similarity giữa hai tập hợp."""
        intersection = len(set(set1) & set(set2))
        union = len(set(set1) | set(set2))
        return float(intersection) / float(union) if union != 0 else 0.0

    def find_similar_dates(self, thresholds):
        """Tìm cặp ngày có Jaccard similarity ≥ s."""
        running_times = {}
        results = {}

        # Định nghĩa UDF cho Jaccard Similarity
        jaccard_udf = udf(self.compute_jaccard, FloatType())

        # Tính thời gian bước pre_process
        start_time = time.time()
        self.pre_process()
        pre_process_time = time.time() - start_time

        print(f"Thời gian tiền xử lý: {pre_process_time:.4f} giây")

        # Vòng lặp qua từng ngưỡng s
        for s in thresholds:
            start_time = time.time()

            # Self-join để tìm tất cả cặp ngày khác nhau
            joined_df = self.transactions.alias("t1").join(
                self.transactions.alias("t2"),
                col("t1.Date") < col("t2.Date")
            )

            # Tính Jaccard Similarity
            similar_dates_df = joined_df.withColumn(
                "JaccardSimilarity", jaccard_udf(col("t1.items"), col("t2.items"))
            ).filter(col("JaccardSimilarity") >= s
            ).select(
                col("t1.Date").alias("Date1"),
                col("t2.Date").alias("Date2"),
                col("JaccardSimilarity")
            )

            # Thời gian thực thi
            runtime = time.time() - start_time + pre_process_time
            running_times[s] = runtime

            num_pairs = similar_dates_df.count()
            results[s] = {
              "df": similar_dates_df,
              "num_pairs": num_pairs
            }

            print(f"s = {s}, số cặp ngày similar = {num_pairs}, thời gian chạy = {runtime:.4f} giây")

        return results, running_times

    def save_results(self, results, output_dir):
        """Lưu kết quả vào file CSV."""
        for s, result in results.items():
            similar_dates_df = result["df"]
            output_file_path = f"{output_dir}/{s}_similar_dates.csv"
            similar_dates_df.write.option("header", "true").mode("overwrite").csv(output_file_path)

    def run(self, thresholds, output_dir):
        """Thực thi toàn bộ quy trình."""
        results, running_times = self.find_similar_dates(thresholds)
        self.save_results(results, output_dir)

        return results, running_times

# ============================ #
#       **CHẠY THỰC THI**
# ============================ #

file_path = f"{working_path}/source/baskets.csv"
output_dir = f"{working_path}/output/bruteforce"
thresholds = [round(i * 0.1, 1) for i in range(11)]  # s từ 0.0 đến 1.0, bước 0.1

# Tạo instance của lớp
brute_force = BruteForceApproach(file_path)

# Chạy phương pháp brute force
results_bruteforce, running_times_bruteforce = brute_force.run(thresholds, output_dir)

# Xem kết quả với ngưỡng s
s = 0.5
print("*"*50)
if s in results_bruteforce:
    print(f"Các cặp đầu tiên có Jaccard similarity ≥ {s}:")
    print(f"Số cặp tìm được: {results_bruteforce[s]['num_pairs']}")
    results_bruteforce[s]["df"].orderBy(col("Date1").asc()).show(20)  # Sắp xếp tăng dần theo Date1

# Dừng SparkSession
brute_force.spark.stop()

Thời gian tiền xử lý: 0.0175 giây
s = 0.0, số cặp ngày similar = 264628, thời gian chạy = 0.1529 giây
s = 0.1, số cặp ngày similar = 264520, thời gian chạy = 0.1997 giây
s = 0.2, số cặp ngày similar = 244672, thời gian chạy = 0.0716 giây
s = 0.3, số cặp ngày similar = 105487, thời gian chạy = 0.0677 giây
s = 0.4, số cặp ngày similar = 10113, thời gian chạy = 0.0904 giây
s = 0.5, số cặp ngày similar = 223, thời gian chạy = 0.0698 giây
s = 0.6, số cặp ngày similar = 3, thời gian chạy = 0.0768 giây
s = 0.7, số cặp ngày similar = 0, thời gian chạy = 0.0854 giây
s = 0.8, số cặp ngày similar = 0, thời gian chạy = 0.0680 giây
s = 0.9, số cặp ngày similar = 0, thời gian chạy = 0.1651 giây
s = 1.0, số cặp ngày similar = 0, thời gian chạy = 0.0857 giây
**************************************************
Các cặp đầu tiên có Jaccard similarity ≥ 0.5:
Số cặp tìm được: 223
+----------+----------+-----------------+
|     Date1|     Date2|JaccardSimilarity|
+----------+----------+-----------------+
|20

# **MinHashLSH**


# MinHashLSH Algorithm


In [ ]:
class MinHashLSHApproach:
    def __init__(self, file_path):
        """Khởi tạo SparkSession và đọc dữ liệu"""
        self.spark = SparkSession.builder.appName("MinHashLSHApproach").getOrCreate()
        self.file_path = file_path
        self.df = self.spark.read.csv(self.file_path, header=True, inferSchema=True)
        self.df = self.df.withColumn("Date", to_date(col("Date"), "dd/MM/yyyy"))

    def shingling(self):
        """Bước 1: Shingling - Chuyển items thành binary vectors"""
        self.pre_process()
        self.cv = CountVectorizer(inputCol="items", outputCol="features", binary=True)
        self.cv_model = self.cv.fit(self.transactions)
        self.featurized_data = self.cv_model.transform(self.transactions)

    def pre_process(self):
        # Tiền xử lý: Nhóm sản phẩm theo ngày
        self.transactions = self.df.groupBy("Date").agg(
            collect_set("itemDescription").alias("items")
        )

    def minhash_lsh(self, num_hash_tables=10, seed=1024):
        """Bước 2 & 3: MinHash + LSH với seed cố định"""
        self.mh = MinHashLSH(
            inputCol="features",
            outputCol="hashes",
            numHashTables=num_hash_tables
        ).setSeed(seed)  # Đặt seed để đảm bảo kết quả ổn định

        self.model = self.mh.fit(self.featurized_data)

    def find_similar_dates(self, thresholds, num_hash_tables=10):
        """Tìm cặp ngày tương đồng với các ngưỡng khác nhau"""
        running_times = {}
        results = {}

        # Thực hiện Shingling -> MinHash -> LSH
        start_time = time.time()
        self.shingling()
        self.minhash_lsh(num_hash_tables)
        pre_process_time = time.time() - start_time

        print(f"Thời gian tiền xử lý: {pre_process_time:.4f} giây")

        for s in thresholds:
            start_time = time.time()

            # Tìm cặp có Jaccard similarity ≥ s (dùng approxSimilarityJoin với JaccardDistance = 1 - similarity)
            similar_pairs = self.model.approxSimilarityJoin(
                self.featurized_data,
                self.featurized_data,
                threshold=1.0 - s,  # Chuyển đổi similarity -> distance
                distCol="JaccardDistance"
            ).filter(
                col("datasetA.Date") < col("datasetB.Date")  # Loại bỏ cặp trùng và đảo ngược
            ).withColumn(
                "JaccardSimilarity", expr("1 - JaccardDistance")  # Chuyển ngược về similarity
            ).select(
                col("datasetA.Date").alias("Date1"),
                col("datasetB.Date").alias("Date2"),
                col("JaccardSimilarity")
            )

            # Thời gian thực thi
            runtime = time.time() - start_time + pre_process_time
            running_times[s] = runtime

            num_pairs = similar_pairs.count()  # Đếm số cặp
            results[s] = {
                "df": similar_pairs,
                "num_pairs": num_pairs
            }

            print(f"s = {s}, số cặp ngày similar = {num_pairs}, thời gian chạy = {runtime:.4f}s")

        return results, running_times

    def save_results(self, results, output_dir):
        """Lưu kết quả vào file CSV."""
        for s, result in results.items():
            similar_dates_df = result["df"]
            output_file_path = f"{output_dir}/{s}_similar_dates.csv"
            similar_dates_df.write.option("header", "true").mode("overwrite").csv(output_file_path)

    def run(self, thresholds, output_dir=None, num_hash_tables=5):
        """Chạy toàn bộ pipeline"""
        results, runtimes = self.find_similar_dates(thresholds, num_hash_tables)
        self.save_results(results, output_dir)

        return results, runtimes

# ============================ #
#       **CHẠY THỰC THI**
# ============================ #

file_path = f"{working_path}/source/baskets.csv"
output_dir = f"{working_path}/output/minhash"
thresholds = [round(i * 0.1, 1) for i in range(11)]  # [0.0, 0.1, ..., 1.0]

# Chạy chương trình với MinHashLSH
minhash = MinHashLSHApproach(file_path)
results_minhash, running_times_minhash = minhash.run(thresholds, output_dir)

# Xem kết quả với ngưỡng s
s = 0.5
print("*"*50)
if s in results_minhash:
    print(f"Các cặp đầu tiên có Jaccard similarity ≥ {s}:")
    print(f"Số cặp tìm được: {results_minhash[s]['num_pairs']}")
    results_minhash[s]["df"].orderBy(col("Date1").asc()).show(20)  # Sắp xếp tăng dần theo Date1

# Dừng SparkSession
minhash.spark.stop()

Thời gian tiền xử lý: 0.6953 giây
s = 0.0, số cặp ngày similar = 261935, thời gian chạy = 0.8755s
s = 0.1, số cặp ngày similar = 261828, thời gian chạy = 0.8059s
s = 0.2, số cặp ngày similar = 240110, thời gian chạy = 0.8393s
s = 0.3, số cặp ngày similar = 102208, thời gian chạy = 0.8513s
s = 0.4, số cặp ngày similar = 8917, thời gian chạy = 0.8866s
s = 0.5, số cặp ngày similar = 134, thời gian chạy = 0.8665s
s = 0.6, số cặp ngày similar = 2, thời gian chạy = 0.8317s
s = 0.7, số cặp ngày similar = 0, thời gian chạy = 0.9070s
s = 0.8, số cặp ngày similar = 0, thời gian chạy = 0.8044s
s = 0.9, số cặp ngày similar = 0, thời gian chạy = 0.7751s
s = 1.0, số cặp ngày similar = 0, thời gian chạy = 0.8420s
**************************************************
Các cặp đầu tiên có Jaccard similarity ≥ 0.5:
Số cặp tìm được: 134
+----------+----------+------------------+
|     Date1|     Date2| JaccardSimilarity|
+----------+----------+------------------+
|2014-01-02|2015-11-10| 0.559322033898305|
|2

# **Biểu đồ so sánh hai Approach**

In [ ]:
# Chuyển kết quả thời gian chạy thành danh sách
s_values = [round(i * 0.1, 1) for i in range(11)]
time_minhash = list(running_times_minhash.values())
time_bruteforce = list(running_times_bruteforce.values())

# ===================== #
#   Biểu đồ thời gian   #
# ===================== #
plt.figure(figsize=(10, 5))
plt.plot(s_values, time_minhash, marker='o', linestyle='-', color='b', label="MinHashLSH")
plt.plot(s_values, time_bruteforce, marker='s', linestyle='-', color='r', label="Brute Force")
plt.xlabel("Jaccard Similarity Threshold (s)")
plt.ylabel("Running Time (seconds)")
plt.title("Comparison of Running Time: MinHashLSH vs Brute Force")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()



In [ ]:
def generate_comparison_table(results_bruteforce, results_minhash, s_values):
    # Tạo dữ liệu cho bảng
    table_data = []
    for s in s_values:
        num_bf = results_bruteforce[s]["num_pairs"]
        num_mh = results_minhash[s]["num_pairs"]
        diff = abs(num_bf - num_mh)

        table_data.append([s, num_bf, num_mh, diff])

    # Định dạng bảng
    headers = ["Threshold (s)", "Brute-force Pairs", "MinHashLSH Pairs", "Difference"]
    table = tabulate(table_data, headers=headers, tablefmt="grid")

    # Thêm tiêu đề
    comparison_title = "=== COMPARISON OF PAIRS FOUND ==="
    full_table = f"{comparison_title}\n{table}"

    return full_table

# Cách sử dụng
s_values = [round(i * 0.1, 1) for i in range(11)]  # [0.0, 0.1, ..., 1.0]
print(generate_comparison_table(results_bruteforce, results_minhash, s_values))

=== COMPARISON OF PAIRS FOUND ===
+-----------------+---------------------+--------------------+--------------+
|   Threshold (s) |   Brute-force Pairs |   MinHashLSH Pairs |   Difference |
+=================+=====================+====================+==============+
|             0   |              264628 |             261935 |         2693 |
+-----------------+---------------------+--------------------+--------------+
|             0.1 |              264520 |             261828 |         2692 |
+-----------------+---------------------+--------------------+--------------+
|             0.2 |              244672 |             240110 |         4562 |
+-----------------+---------------------+--------------------+--------------+
|             0.3 |              105487 |             102208 |         3279 |
+-----------------+---------------------+--------------------+--------------+
|             0.4 |               10113 |               8917 |         1196 |
+-----------------+-----------